# Analyse  Source Table Visit

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/03/05
- update : 2024/03/05


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

## Config

In [ ]:


file_selected_visits = "../data/202402/SelectedVisits_fall2023_tract_3864.csv"
filesource_in ="../data/202402/sourceTable_visit_202402_20231121-20231222.csv"
#filesource_in ="../data/202402/sourceTable_visit_202402.csv

## Read

### read visits

In [ ]:
df_visit = pd.read_csv(file_selected_visits,index_col=0)

In [ ]:
df_visit.head()

### Read sourceTable_visit

In [ ]:
df = pd.read_csv(filesource_in,index_col=0)

In [ ]:
df.head()

In [ ]:
df.dropna(axis=1, how='all',inplace =True)

In [ ]:
df["nightObs"] =df.apply(lambda x: x['visitId']//100_000, axis=1)

In [ ]:
df_sel = df[['parentSourceId','ccdVisitId','x','y','ra','dec','coord_ra','coord_dec','calibFlux','psfFlux','gaussianFlux','localPhotoCalib','visit','band','extendedness_flag','detect_isPrimary']]

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
df_sel["psfMag"] =  df_sel['psfFlux'].map(convert_fluxtomag)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(lambda x:-2.50 * np.log10(x) + 31.4)
df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(convert_fluxtomag)

In [ ]:
df_sel = df_sel[df_sel['extendedness_flag']==False & df_sel['detect_isPrimary']]

In [ ]:
df_sel.dropna(axis=0,inplace = True)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
df_sel.plot.scatter('ra','dec',c='ccdVisitId',ax=ax,colormap='jet',grid=True)
ax.set_aspect("auto")

In [ ]:
df_sel = df_sel[df_sel["dec"]>-40]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
df_sel.plot.scatter('coord_ra','coord_dec',c='ccdVisitId',colormap='jet',grid=True,ax=ax)
ax.set_aspect("auto")

## Mag distributions

In [ ]:
all_bands = ["g","r","i","z","y"]
NB = len(all_bands)
band_color = ["g","r","orange","grey","k"]
all_df_perband = [ df_sel[df_sel['band'] == band] for band in all_bands ]
fig,axs = plt.subplots(1,NB,figsize=(16,4))
for iband in range(NB):
    all_df_perband[iband].hist("psfMag",bins=50,range=(10,22),ax=axs[iband],facecolor=band_color[iband],label=all_bands[iband])
plt.tight_layout()


In [ ]:
all_bands = ["g","r","i","z","y"]
NB = len(all_bands)
band_color = ["g","r","orange","grey","k"]
all_df_perband = [ df_sel[df_sel['band'] == band] for band in all_bands ]
fig,axs = plt.subplots(1,NB,figsize=(16,4))
for iband in range(NB):
    all_df_perband[iband].hist("localPhotoCalib",bins=50,ax=axs[iband],facecolor=band_color[iband],label=all_bands[iband])
plt.tight_layout()


## Merge sourceTable_visit with visitTable 

In [ ]:
df_m = df_sel.join(df_visit,on="visit",lsuffix='_left', rsuffix='_right')

In [ ]:
df_m

In [ ]:
df_m.dropna(axis=0,how='any')